In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


In [2]:
# Specify the file path
file_path_test = "unlabelled_test_data.csv"
file_path_training = "training_data.csv"

# Read the CSV file
unlabelled_test_data = pd.read_csv(file_path_test)
training_data = pd.read_csv(file_path_training)

In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Using TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000)
X = vectorizer.fit_transform(training_data['sentence'])
y = training_data['difficulty']

In [4]:
# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)


In [5]:
from sklearn.linear_model import LogisticRegression

# Initialize and train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

LogisticRegression(max_iter=1000)

In [6]:
from sklearn.metrics import classification_report, accuracy_score

# Predict on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(classification_rep)

Accuracy: 0.4625
Classification Report:
              precision    recall  f1-score   support

          A1       0.51      0.68      0.58       166
          A2       0.37      0.33      0.35       158
          B1       0.44      0.31      0.37       166
          B2       0.46      0.43      0.44       153
          C1       0.44      0.44      0.44       152
          C2       0.51      0.57      0.54       165

    accuracy                           0.46       960
   macro avg       0.46      0.46      0.45       960
weighted avg       0.46      0.46      0.45       960



In [7]:
# Step 1: Predict the difficulty levels
X_unlabelled = vectorizer.transform(unlabelled_test_data['sentence'])
predicted_difficulties = logistic_model.predict(X_unlabelled)

# Step 2: Create a new DataFrame
predictions_df = pd.DataFrame({
    'id': unlabelled_test_data['id'],
    'predicted_difficulty': predicted_difficulties
})

# Step 3: Export to CSV
output_file_path = 'predicted_difficulties.csv'  # You can change the file name as needed
predictions_df.to_csv(output_file_path, index=False)

In [8]:
#I want to extract some the features from the training data so I can use them to train my model
#training_data['type_token_ratio'] - This ratio compares the number of unique words (types) to the total number of words (tokens) in a text.
#training_data['syntactic_complexity'] - counts commas as a proxy for complexity.

In [9]:
import numpy as np
import nltk
nltk.download('punkt')

def sentence_length(sentence):
    return len(sentence.split())

def average_word_length(sentence):
    words = sentence.split()
    return np.mean([len(word) for word in words]) if words else 0

def type_token_ratio(sentence):
    words = sentence.split()
    return len(set(words)) / len(words) if words else 0

def syntactic_complexity(sentence):
    return sentence.count(',')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/oanaalexandrablanc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
training_data['sentence_length'] = training_data['sentence'].apply(sentence_length)
training_data['avg_word_length'] = training_data['sentence'].apply(average_word_length)
training_data['type_token_ratio'] = training_data['sentence'].apply(type_token_ratio)
training_data['syntactic_complexity'] = training_data['sentence'].apply(syntactic_complexity)

In [11]:
# Transform the text data using TF-IDF
X_text_features = vectorizer.transform(training_data['sentence'])

# Convert the linguistic features into a format that can be concatenated with the TF-IDF features
import scipy.sparse as sp
X_linguistic_features = sp.csr_matrix(training_data[['sentence_length', 'avg_word_length', 'type_token_ratio', 'syntactic_complexity']])

# Combine the TF-IDF features with the new linguistic features
X_combined = sp.hstack([X_text_features, X_linguistic_features])

In [12]:
# Split the combined features into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Train the Logistic Regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

/Users/oanaalexandrablanc/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=1000)

In [13]:
# Predict on the test set
y_pred = logistic_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print(classification_rep)


Accuracy: 0.5
              precision    recall  f1-score   support

          A1       0.58      0.80      0.67       166
          A2       0.44      0.41      0.43       158
          B1       0.49      0.39      0.43       166
          B2       0.48      0.48      0.48       153
          C1       0.44      0.43      0.43       152
          C2       0.52      0.48      0.50       165

    accuracy                           0.50       960
   macro avg       0.49      0.50      0.49       960
weighted avg       0.49      0.50      0.49       960



In [14]:
#extract the same features for the unlabelled test data

In [15]:
unlabelled_test_data['sentence_length'] = unlabelled_test_data['sentence'].apply(sentence_length)
unlabelled_test_data['avg_word_length'] = unlabelled_test_data['sentence'].apply(average_word_length)
unlabelled_test_data['type_token_ratio'] = unlabelled_test_data['sentence'].apply(type_token_ratio)
unlabelled_test_data['syntactic_complexity'] = unlabelled_test_data['sentence'].apply(syntactic_complexity)

# Transform the sentences using the same TF-IDF vectorizer
X_unlabelled_text = vectorizer.transform(unlabelled_test_data['sentence'])
X_unlabelled_linguistic = sp.csr_matrix(unlabelled_test_data[['sentence_length', 'avg_word_length', 'type_token_ratio', 'syntactic_complexity']])
X_unlabelled_combined = sp.hstack([X_unlabelled_text, X_unlabelled_linguistic])

# Step 3: Predict the difficulty levels
predicted_difficulties = logistic_model.predict(X_unlabelled_combined)

# Step 4: Create a new DataFrame with predictions
predictions_df = pd.DataFrame({
    'id': unlabelled_test_data['id'],
    'predicted_difficulty': predicted_difficulties
})

# Step 5: Export to CSV
predictions_df.to_csv('predicted_difficulties2.csv', index=False)

In [16]:
#Now I'd like to add a readability score as a feature for my model. I'll use spaCy

In [17]:
#import spacy

#nlp = spacy.load("fr_core_news_sm")

In [18]:
#def extract_features(sentence):
#    doc = nlp(sentence)
#    num_clauses = sum(1 for _ in doc.sents)  # Sentence Complexity: Measured by the number of clauses or parse tree depth.
#    pos_counts = doc.count_by(spacy.attrs.POS)  # Part-of-Speech Usage: Frequency of certain parts of speech.
#    num_ner = len(doc.ents)  # Named Entity Recognition (NER): Presence of named entities.

    # Example: return the number of clauses and named entities
#    return num_clauses, num_ner

# Apply the feature extraction to each sentence
#training_data['num_clauses'], training_data['num_ner'] = zip(*training_data['sentence'].apply(extract_features))

In [19]:
#Include the new features into the  model

In [20]:
#from scipy.sparse import hstack

# First, vectorize the textual data using TF-IDF
#X_text_features = vectorizer.transform(training_data['sentence'])

# Then, prepare the spaCy-based features
#X_spacy_features = training_data[['num_clauses','num_ner']].values

# Combine the TF-IDF features with the spaCy features
#X_combined = hstack([X_text_features, X_spacy_features])

In [21]:
#Retrain the model

In [22]:
# Split the data into training and test sets
#X_train, X_test, y_train, y_test = train_test_split(X_combined, training_data['difficulty'], test_size=0.2, random_state=42)

# Train the logistic regression model
#logistic_model = LogisticRegression(max_iter=1000)
#logistic_model.fit(X_train, y_train)

In [23]:
# Evaluate the model
#y_pred = logistic_model.predict(X_test)
#accuracy = accuracy_score(y_test, y_pred)
#classification_rep = classification_report(y_test, y_pred)

#print(f"Accuracy: {accuracy}")
#print(classification_rep)

In [24]:
#The accuracy is worse than before...

In [25]:
#Now I'll try text embeddings
#CamemBERT -> pip install transformers torch, pip install sentencepiece

In [26]:
from transformers import CamembertModel, CamembertTokenizer
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Compute additional features and add them to the DataFrame
training_data['sentence_length'] = training_data['sentence'].apply(sentence_length)
training_data['avg_word_length'] = training_data['sentence'].apply(average_word_length)
training_data['type_token_ratio'] = training_data['sentence'].apply(type_token_ratio)
training_data['syntactic_complexity'] = training_data['sentence'].apply(syntactic_complexity)

# Load tokenizer and model for CamemBERT
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base")

# Tokenize and encode sentences in the dataset
inputs = tokenizer(list(training_data['sentence']), padding=True, truncation=True, max_lenght= 128, return_tensors="pt")
input_ids = inputs['input_ids']

# Create a DataLoader for batch processing
batch_size = 8  # Adjust based on your system's capability
dataset = TensorDataset(input_ids)
dataloader = DataLoader(dataset, batch_size=batch_size)

# Generate embeddings in batches
embeddings = []
model.eval()
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch[0]
        outputs = model(input_ids)
        batch_embeddings = outputs.last_hidden_state.mean(dim=1)
        embeddings.append(batch_embeddings)
embeddings = torch.cat(embeddings, dim=0).numpy()

# Combine CamemBERT embeddings with additional features
combined_features = np.hstack((embeddings, 
                               training_data[['sentence_length', 'avg_word_length', 'type_token_ratio', 'syntactic_complexity']].values))

# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(combined_features, training_data['difficulty'], test_size=0.2)

# Train logistic regression model
logistic_model = LogisticRegression(max_iter=1000)
logistic_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = logistic_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


: 